<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import psutil
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
import seaborn as sns
import papermill
import umap

/home/marcelino.maita/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(papermill.__version__)

2.6.0


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [3]:
DATA_PATH ="../data"
path_lb_embb = os.path.join(DATA_PATH, "embbedings_khipu/lb_khipu_A.npy")
path_dataset = os.path.join(DATA_PATH, "spotify_dataset_sin_duplicados_4.csv")

# Para hacer pruebitas y luego mandarlo como sbatch
TESTING = True
_EMBEDDINGS = np.load(path_lb_embb)

if TESTING:
    _NROWS = 1000
    _EMBEDDINGS = _EMBEDDINGS[:1000]
    _EPHOCS = 1000
else:
    _NROWS = None
    _EPHOCS = 10000
    

df = pd.read_csv(path_dataset, nrows = _NROWS)
df['Explicit_binary'] = df['Explicit'].map({'Yes': 1, 'No': 0})

X = _EMBEDDINGS

y = df['Explicit_binary']



X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42 
)

y_train = y_train.values.reshape(-1, 1).astype(np.float32)
y_test = y_test.values.reshape(-1, 1).astype(np.float32)


def seeRam():
    process = psutil.Process(os.getpid())
    print(f"RAM usada (MB): {process.memory_info().rss / (1024 * 1024):.2f}")

print(X.shape)
print(y.shape)
seeRam()

FileNotFoundError: [Errno 2] No such file or directory: '../data/embbedings_khipu/lb_khipu_A.npy'

In [ ]:
# CODE HERE for backward pass

# f0, f1, f2
class Linear:
    def __init__(self, in_dim, out_dim):
        self.W = np.random.randn(in_dim, out_dim) * np.sqrt(2.0 / in_dim)
        self.b = np.zeros((1, out_dim))
    def forward(self, x):
        self.x = x
        return x @ self.W + self.b

    def backward(self, grad_output):
        self.grad_W = self.x.T @ grad_output
        self.grad_b = np.sum(grad_output, axis=0, keepdims=True)
        return grad_output @ self.W.T

# h0, h1, h2, ....
class ReLU:
    def forward(self, x):
        self.mask = x > 0
        return x * self.mask
    def backward(self, grad_output):
        return grad_output * self.mask

class Sigmoid:
    def forward(self, x):
        self.out = 1 / (1 + np.exp(-x))
        return self.out

    def backward(self, grad_output):
        return grad_output * self.out * (1 - self.out)

# error
class MSELoss:
    def forward(self, pred, target):
        self.diff = pred - target
        return np.mean(self.diff ** 2)
    def backward(self):
        return 2.0 * self.diff / self.diff.shape[0]

In [ ]:
class MLP:
    def __init__(self, dims):
        self.layers = []
        for i in range(len(dims)-2):
            self.layers.append(Linear(dims[i], dims[i+1])) #fi
            self.layers.append(ReLU()) # hi

        self.layers.append(Linear(dims[-2], dims[-1]))

    # CODE HERE (forward, backward pass)
    def forward(self, x):
        for l in self.layers:
            x = l.forward(x)
        return x

    def backward(self, grad):
        for l in reversed(self.layers):
            grad = l.backward(grad)

    @property
    def n_params(self):
        total = 0
        for l in self.layers:
            if isinstance(l, Linear):
                total += l.W.size + l.b.size
        return total

    def train(self, X, y, lr=0.1, epochs=10000, print_every=100):
      loss_fn = MSELoss()
      history = []
      for epoch in range(epochs):
          #forward
          pred = self.forward(X)
          loss = loss_fn.forward(pred, y)
          #backward
          grad = loss_fn.backward()
          self.backward(grad)

          # Descenso de la gradiente
          for layer in self.layers:
              if isinstance(layer, Linear):
                  layer.W -= lr * layer.grad_W
                  layer.b -= lr * layer.grad_b
          history.append(loss)
          if epoch % print_every == 0:
              print(f"Epoch {epoch}, Loss: {loss:.6f}")
      return history

    def train_batches(self, X, y, lr=0.1, epochs=10000, batch_size=32, print_every=100):
        loss_fn = MSELoss()
        history = []

        n = X.shape[0]
        for epoch in range(epochs):
            # Barajamos los datos al inicio de cada época
            perm = np.random.permutation(n)
            X_shuffled = X[perm]
            y_shuffled = y[perm]

            for i in range(0, n, batch_size):
                xb = X_shuffled[i:i+batch_size]
                yb = y_shuffled[i:i+batch_size]

                # Forward
                pred = self.forward(xb)
                loss = loss_fn.forward(pred, yb)

                # Backward
                grad = loss_fn.backward()
                self.backward(grad)

                # Actualización
                for layer in self.layers:
                    if isinstance(layer, Linear):
                        layer.W -= lr * layer.grad_W
                        layer.b -= lr * layer.grad_b

            pred_full = self.forward(X)
            epoch_loss = loss_fn.forward(pred_full, y)
            history.append(epoch_loss)

            if epoch % print_every == 0:
                print(f"Epoch {epoch}, Loss: {epoch_loss:.6f}")

        return history


In [ ]:
print(X_train.shape)
# y_train = Y.values.reshape(-1, 1).astype(np.float32) 
print(y_train.shape)

In [ ]:
mlp = MLP([300, 32,32, 1])
history= mlp.train(X_train, y_train, lr=0.5, epochs=_EPHOCS, print_every=1000)
print('Parámetros totales:', mlp.n_params)


In [ ]:
# Perdida
plt.figure()
plt.plot(history)
plt.xlabel("Épocas")
plt.ylabel("Error")
plt.title("Evolución de la pérdida (XOR)")
plt.grid(True)
plt.show()

In [ ]:
# Pasando test a la red
pred_test = mlp.forward(X_test)

# Se hace la predicción
pred_labels = (pred_test >= 0.5).astype(int).flatten()
y_test_true = y_test.flatten()

# Calcular Accuracy y Reporte
from sklearn.metrics import accuracy_score, classification_report

print(f"Accuracy en Test: {accuracy_score(y_test_true, pred_labels):.4f}")
print(classification_report(y_test_true, pred_labels))


In [ ]:
cm = confusion_matrix(y_test_true, pred_labels)

plt.figure(figsize=(6,5))
print(cm)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Not Explicit', 'Explicit'], yticklabels=['Not Explicit', 'Explicit'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()